<a href="https://colab.research.google.com/github/NeerajNair8/Priority_App/blob/master/Test_multiple_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cropping the lips from MIRACL VCI dataset

 Mounting The drive for Permanent memory 

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')
#drive.flush_and_unmount()

Mounted at /content/gdrive


Uploading the kaggle API key to access the database 

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"neerajs7","key":"b6b2be0e57ff1320360d806c7247e15a"}'}

Downloading and unzipping the dataset 

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d apoorvwatsky/miraclvc1
! unzip miraclvc1.zip
! rm -rf miraclvc1.zip
#! cp -r shape_predictor_68_face_landmarks.dat /content/gdrive/"My Drive"/

In [ ]:
!rm -r dataset
!rm -r cropped

Necessary Imports

In [6]:
import os
from PIL import Image
import numpy as np
import sys
import cv2
import dlib
import imutils
from imutils import face_utils
from multiprocessing import Pool
from multiprocessing import cpu_count
import math
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import load_img               
from tensorflow.keras.preprocessing.image import img_to_array 
import os
from shutil import copyfile,rmtree
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.callbacks import Callback,EarlyStopping,ModelCheckpoint
from keras.backend import clear_session
import gc

print("Executed")

Executed


Using Dlib's face detector to detect the face in the image  
Then using dlib's shape predictor different landmarks on the face can be predicted

In [ ]:
detector  = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/gdrive/My Drive/shape_predictor_68_face_landmarks.dat")

Directory Structure:  


Cropped  
>People  
>>Data Types(Word,Phrase)    
>>>Instances(10)   
>>>>Word/Phrase index  
>>>>>Images     

Defining the people to be used for training, testing and validation... and other required variables

In [7]:
from multiprocessing import cpu_count

# Pre-defined labels
allPeople        = ['F01','F02','F04','F05','F06', 'F07','F08','F09','F10','F11','M01','M02','M04','M07','M08']
#allPeople        = ['F01','F02','F04','F05','F06']
trainPeople      = ['F01','F02','F04','F05','F07','F08','F10','F11','M01','M02','M04','M07'] # 2400
validPeople      = ['F06','M08'] # 400
testPeople       = ['F09'] # 200
data_types       = ['words','phrases']
folder_enum      = ['01','02','03','04','05','06','07','08','09','10']
image_dir        = "/content/gdrive/My Drive/dataset/dataset"



# for one hot encoding in numbers word will mapped to a label value between 0 to 9 & for phrases to 10 to 19

dict_mapping    = {
                    0:"Begin",
                    1:"Choose",
                    2:"Connection",
                    3:"Navigation",
                    4:"Next",
                    5:"Previous",
                    6:"Start",
                    7:"Stop",
                    8:"Hello",
                    9:"Web",
                    10:"Stop navigation", 
                    11:"Excuse me",
                    12:"I am sorry", 
                    13:"Thank you", 
                    14:"Good bye", 
                    15:"I love this game",
                    16:"Nice to meet you",
                    17:"You are welcome",
                    18:"How are you",
                    19:"Have a good time"
                  }

max_image_len = 0
max_image_wid = 0
max_img_seq = 0

avg_image_len = 0
avg_image_wid = 0

tot_images = 0

MAX_WIDTH      = 640
MAX_HEIGHT     = 480
max_seq_length = 25  # max number of frames mapped to a word/phrase


# MultiProcessing 
numCores   = cpu_count()
procIDs    = list(range(0, numCores))
globCount  = 0

Recreating the Same Directory structure as MIRACL VC1

In [ ]:
lipsDir = '/content/gdrive/My Drive/Neeraj_temp/cropped'
if not os.path.exists(lipsDir):
  os.mkdir(lipsDir)
for i in allPeople:
  if not os.path.exists(lipsDir+'/'+i):
    os.mkdir(lipsDir+'/'+i)
  for j in data_types:
    if not os.path.exists(lipsDir+'/'+i+"/"+j):
      os.mkdir(lipsDir+"/"+i+"/"+j)
    for k in folder_enum:
      if not os.path.exists(lipsDir+'/'+i+"/"+j+"/"+k):
        os.mkdir(lipsDir+"/"+i+"/"+j+"/"+k)

*Extracting Lips and cropping  
*Lips and a small portion of its surrounding area is extracted from the image and resized to (90,90)  
*These are stored as new images in the same directory structure as miracl vc1   
*These images are stored in mounted google drive to be used later.

In [ ]:
def extractLips(img):
  try:
    rects = detector(img, 1)
    shape = predictor(img, rects[0])
    shape = face_utils.shape_to_np(shape)
    
    (x, y, w, h) = cv2.boundingRect(np.array([shape[48:68]])) # extracting the lips
    roi = img[y-15:y+15 + h, x-15:x + w+15]
    res = cv2.resize(roi, dsize=(90, 90), interpolation=cv2.INTER_CUBIC)
  
  except:
    return []
  
  return res
def preProcessing( peopleList):
  data_dir = '/content/gdrive/My Drive/Neeraj_temp/cropped'
  #allPeople = allPeople[0]
  
  
  personID = peopleList
  instanceID = 0
  error_count = 0
  for dataType in data_types:
    for dataTypeID in folder_enum:
      for instanceID in folder_enum:
        path       = os.path.join(image_dir, personID, dataType, dataTypeID, instanceID)
        filelist   = sorted(os.listdir(path + '/'))
        sequence = []
        if not os.path.exists(lipsDir+"/"+personID+"/"+dataType+"/"+dataTypeID+"/"+instanceID+"/"):
          os.mkdir(lipsDir+"/"+personID+"/"+dataType+"/"+dataTypeID+"/"+instanceID+"/")
        number = 0
        for img_name in filelist:
          number += 1
          if img_name.startswith('color'):
            
            image   = load_img(path + '/' + img_name,target_size = (480, 640))
            image   = img_to_array(image)
            image   = image.astype(np.uint8)
            imgLips = extractLips(image)

            #os.remove(path + '/' + img_name)
            
            if(imgLips == []):
              error_count += 1
              print("Face Error : ",error_count)
              continue

            save_path  = lipsDir+"/"+personID+"/"+dataType+"/"+dataTypeID+"/"+instanceID+"/"
            im = Image.fromarray(imgLips)
            im.save(save_path + "cropped" +str(number)+".jpg")
            print(personID+"/"+dataType+"/"+dataTypeID+"/"+instanceID+"/" + "cropped" +str(number)+".jpg")
            #sequence.append(imgLips)
            
       # instance_index += 1
  

Since operation on each folder is independant on each other, we can process them parallelly.  
This can be done using the multiprocessing module in python.  
Colab provides a dual core cpu and both the cores are used for this operation

In [ ]:
from multiprocessing import Pool

#args = [(allPeople), X_train, y_train, X_val, y_val, X_test, y_test]

print("Launching pool using {} processes...".format(cpu_count()))
pool = Pool(processes=cpu_count())

pool.map(preProcessing, allPeople)
# close the pool and wait for all processes to finish
print("[INFO] waiting for processes to finish...")
pool.close()
pool.join()

## Creating a modified dataset for training with RNNs

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Importing the required libraries

In [ ]:
import os
from PIL import Image
import numpy as np
import sys
import cv2
import dlib
import imutils
import math
from matplotlib import pyplot as plt
from shutil import copyfile,rmtree
print("Executed")

Defining the people to be used for training, testing and validation... and other required variables

In [ ]:
from multiprocessing import cpu_count

# Pre-defined labels
allPeople        = ['F01','F02','F04','F05','F06', 'F07','F08','F09','F10','F11','M01','M02','M04','M07','M08']
#allPeople        = ['F01','F02','F04','F05','F06']
trainPeople      = ['F01','F02','F04','F05','F07','F08','F10','F11','M01','M02','M04','M07'] # 2400
validPeople      = ['F06','M08'] # 400
testPeople       = ['F09'] # 200
data_types       = ['words','phrases']
folder_enum      = ['01','02','03','04','05','06','07','08','09','10']
image_dir        = "/content/gdrive/My Drive/dataset/dataset"


# for one hot encoding in numbers word will mapped to a label value between 0 to 9 & for phrases to 10 to 19

dict_mapping    = {
                    0:"Begin",
                    1:"Choose",
                    2:"Connection",
                    3:"Navigation",
                    4:"Next",
                    5:"Previous",
                    6:"Start",
                    7:"Stop",
                    8:"Hello",
                    9:"Web",
                    10:"Stop navigation", 
                    11:"Excuse me",
                    12:"I am sorry", 
                    13:"Thank you", 
                    14:"Good bye", 
                    15:"I love this game",
                    16:"Nice to meet you",
                    17:"You are welcome",
                    18:"How are you",
                    19:"Have a good time"
                  }

max_image_len = 0
max_image_wid = 0
max_img_seq = 0

avg_image_len = 0
avg_image_wid = 0

tot_images = 0

MAX_WIDTH      = 640
MAX_HEIGHT     = 480
max_seq_length = 25  # max number of frames mapped to a word/phrase


# MultiProcessing 
numCores   = cpu_count()
procIDs    = list(range(0, numCores))
globCount  = 0

New directory structure for training  

>Modified_Dataset
>>Folders(Train,test,Validation)
>>>Label( 1 to 19 )
>>>>Cropped Images    



In [ ]:
#id_folders = ['00']
if not os.path.exists('./Modified_Dataset'):
  os.mkdir('./Modified_Dataset')
#os.chdir('./Modified_Dataset')
folders = ['Train','Validation','Test']
for folder in folders:
  if not os.path.exists("./Modified_Dataset"+'/'+folder):
    os.mkdir("./Modified_Dataset"+'/'+folder)
  for key in dict_mapping:
    if not os.path.exists("./Modified_Dataset"+'/'+folder+'/'+str(key)):
      os.mkdir("./Modified_Dataset"+'/'+folder+'/'+str(key))

These images were stored as numpy arrays to be quickly accessed for training.
To make the size of the sequences same, every instance is truncated at 20 images if it contains more than 20 images. If the instance contains less than 20 images, a pad array of zeros is appended to the end of the array.  
Each array is of shape (20,90,90,3)

In [ ]:
lipsDir = '/content/cropped'

train_id = 0
validation_id = 0
test_id = 0
max_sequence_length = 20
pad_array = [np.zeros((90, 90,3))]

def get_savepath(personID,dataTypeID):
  global train_id,validation_id,test_id
  train_path = "/content/Modified_Dataset/Train"
  validation_path = "/content/Modified_Dataset/Validation"
  test_path = "/content/Modified_Dataset/Test"
  if personID in trainPeople:
    save_path = train_path + '/' + str(dataTypeID) + '/' + str(train_id)
    train_id += 1
  elif personID in validPeople:
    save_path = validation_path + '/' + str(dataTypeID) + '/' + str(validation_id)
    validation_id += 1
  else :
    save_path = test_path + '/' + str(dataTypeID) + '/' + str(test_id)
    test_id += 1
  return save_path

for personID in allPeople:
  for dataType in data_types:
    for dataTypeID in folder_enum:
      for instanceID in folder_enum:
        path       = os.path.join(lipsDir, personID, dataType, dataTypeID, instanceID)
        save_path  = None
        filelist   = sorted(os.listdir(path + '/'))
        sequence = []
        for img_name in filelist:           
          image   = load_img(path + '/' + img_name,target_size = (90, 90))
          image   = img_to_array(image)
          #image   = image.astype(np.uint8)
          sequence.append(image)
        if len(sequence)>20:
          sequence = sequence[:20]
        sequence.extend(pad_array*(max_sequence_length-len(sequence))) 
        word_label = int(dataTypeID)
        if dataType == "words":
          save_path = get_savepath(personID,word_label-1)
        else:
          save_path = get_savepath(personID,word_label+10-1)

        array = np.array(sequence)
        np.save(save_path,array)
        print(save_path,os.path.exists(save_path+'.npy'),array.shape)
print('finished')

In [ ]:
print('Training sameples : ',train_id)
print('Validation sameples : ',validation_id)
print('Test sameples : ',test_id)

Training sameples :  2400
Validation sameples :  400
Test sameples :  200


In [ ]:
def normalize_it(X):
  v_min = X.min(axis=(1, 2), keepdims=True)
  v_max = X.max(axis=(1, 2), keepdims=True)
  X = (X - v_min)/((v_max - v_min)+1)
  X = np.nan_to_num(X)
  return X
dir = '/content/gdrive/My Drive/Modified_Dataset'
for folder in os.listdir(dir):
  for class_ in os.listdir(os.path.join(dir,folder)):
    for files in os.listdir(os.path.join(dir,folder,class_)):
      path = os.path.join(dir,folder,class_,files)
      array = np.load(path)
      array = normalize_it(array)
      np.save(path,array)

In [ ]:
!cp -r  /content/gdrive/"My Drive"/Modified_Dataset ./

## Training

In [2]:
import os
from PIL import Image
import numpy as np
import sys
import cv2
import dlib
import imutils
from imutils import face_utils
from multiprocessing import Pool
from multiprocessing import cpu_count
import math
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import load_img               
from tensorflow.keras.preprocessing.image import img_to_array 
import datetime
from shutil import copyfile,rmtree
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from keras.backend import clear_session
import gc

print("Executed")

Executed


In [3]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
#!pip install gdown 
"""!gdown https://drive.google.com/uc?id=1c8Ctqs5OeRFbkbDyeuDQRCeFKA8hWBYA # (20,90,90,3) color arrays
!unzip Dataset.zip
!rm Dataset.zip"""
!gdown https://drive.google.com/uc?id=136gi1k-jelwYbUPz0XSiacqpN927Y2Mf # (20,224,224)
!unzip Dataset_gray.zip
!rm Dataset_gray.zip

Downloading...
From: https://drive.google.com/uc?id=136gi1k-jelwYbUPz0XSiacqpN927Y2Mf
To: /content/Dataset_gray.zip
12.1GB [04:29, 44.7MB/s]
Archive:  Dataset_gray.zip
   creating: content/Modified_Dataset_224_gray/
   creating: content/Modified_Dataset_224_gray/Test/
   creating: content/Modified_Dataset_224_gray/Test/16/
  inflating: content/Modified_Dataset_224_gray/Test/16/161.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/165.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/167.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/169.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/168.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/164.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/163.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/160.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/162.npy  
  inflating: content/Modified_Dataset_224_gray/Test/16/166.npy  
   creating: content/Modi

Creating custom generator to obtain data sequence for training from MIRACL VCI 


In [5]:
from keras.utils import Sequence

class custom_generator(Sequence):
  def __init__(self,startpath,batch_size,input_shape=(224,224),n_channels=3,num_classes=20,shuffle=True,sequence_length=20):
    self.batch_size = batch_size
    self.sequence_length = sequence_length
    self.startpath = startpath
    self.n_channels = n_channels
    self.input_shape = input_shape
    self.all_paths_and_labels = self.get_all_files(self.startpath)
    self.num_classes = num_classes
    self.shuffle = shuffle
    self.on_epoch_end()
    self.flag = True
  def get_all_files(self,startpath):
    all_classes = sorted(os.listdir(startpath))
    all_paths_and_labels = []
    labels = [i for i in range(len(all_classes))]
    self.indices = 0
    for class_,label in zip(all_classes,labels):
      for instance in os.listdir(startpath+'/'+class_):
        all_paths_and_labels.append((os.path.join(startpath,class_,instance),label))
        self.indices += 1
    return all_paths_and_labels
  def on_epoch_end(self):
    self.index = np.arange(self.indices)
    if self.shuffle == True:
      np.random.shuffle(self.index)
  def __len__(self):
    return self.indices//self.batch_size # returns number of batches per epoch
  def normalize_it(self,X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    X = (X - v_min)/((v_max - v_min)+1)
    X = np.nan_to_num(X)
    return X
  def __get_data(self,batch):
    X = np.empty((self.batch_size,self.sequence_length, *self.input_shape, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)
    for i,path_label in enumerate(batch):
      path,label = path_label
      array = np.load(path,allow_pickle=True)
      if array.shape == (self.sequence_length,self.input_shape[0],self.input_shape[1]):
        array = np.expand_dims(array,axis=3)
        X[i,] = array
      else :
        for j in range(self.sequence_length):
          image = array[j,]
          try:
            X[i,j,] = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
          except:
            X[i,j,] = np.ones((224,224,3))*255
            self.error_count += 1
            print("Error reading: ",self.error_count,path)
      #X[i,] = array
      y[i] = label
    X = self.normalize_it(X)
    if self.flag==True:
      pass
      #print("Batch size: ",len(X))
      #print(X[0].shape)
    return X,to_categorical(y,self.num_classes)
  def __getitem__(self,batch_no):
    indexes = self.index[batch_no*self.batch_size:(batch_no+1)*self.batch_size]
    batch = [self.all_paths_and_labels[i] for i in indexes]
    X,y = self.__get_data(batch)
    return X,y

In [6]:
! cp '/content/gdrive/My Drive/mobile_celebrity_val_acc_95_37.h5' ./
! cp "/content/gdrive/My Drive/vgg_celebrity_fine_tune_val_acc_96_07.h5" ./
! cp "/content/gdrive/My Drive/dense_celebrity_fine_tune_val_acc_98_27.h5" ./
! ls

content
dense_celebrity_fine_tune_val_acc_98_27.h5
gdrive
mobile_celebrity_val_acc_95_37.h5
sample_data
vgg_celebrity_fine_tune_val_acc_96_07.h5


In [7]:
model_paths = ['/content/dense_celebrity_fine_tune_val_acc_98_27.h5','/content/mobile_celebrity_val_acc_95_37.h5','/content/vgg_celebrity_fine_tune_val_acc_96_07.h5']
model_names = ["Dense",'Mobilenet','VGG']
cnn_models = {}
for path,name in zip(model_paths,model_names):
  model = load_model(path)
  model.trainable =False
  cnn_models[name] = model

In [8]:
cnn_models['Mobilenet'].input.shape

TensorShape([None, 224, 224, 3])

In [9]:
cnn_models

{'Dense': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fa2004879e8>,
 'Mobilenet': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fa2005fe128>,
 'VGG': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fa20046b6d8>}

In [ ]:
def train_multiple_models(cnn_models):
  if not os.path.exists('/content/gdrive/My Drive/Neeraj_temp'+'/'+'29_10_2020'):
    os.mkdir('/content/gdrive/My Drive/Neeraj_temp'+'/'+'29_10_2020')
  path = '/content/gdrive/My Drive/Neeraj_temp'+'/'+'29_10_2020'
  
  for key in cnn_models:

    clear_session()

    base_dataset_path = '/content/content/Modified_Dataset_224_gray' # (20,224,224)
    #base_dataset_path = '/content/content/Modified_Dataset' # (20,90,90,3)
    train_generator = custom_generator(base_dataset_path+'/'+"Train",32)
    validation_generator = custom_generator(base_dataset_path+'/'+'Validation',32)
    test_generator = custom_generator(base_dataset_path+'/'+"Test",32)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    mc = ModelCheckpoint(path+'/'+'best_model_{}.h5'.format(key), monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

    file = open(path+'/'+'Report{}.txt'.format(key),'w')
    pretrained = cnn_models[key]
    model = Sequential([TimeDistributed(pretrained),
                        TimeDistributed(Flatten()),
                        # LSTMS - Recurrent Network Layer
                        #Bidirectional(LSTM(32, return_sequences=True)),
                        #LSTM(32, return_sequences=True),
                        LSTM(32),
                        Dropout(0.5),
                        Flatten(),

                        # # FC layers group
                        Dense(2048, activation='relu'),
                        Dropout(.5),
                        Dense(1024, activation='relu'),
                        Dropout(.5),
                        Dense(20, activation='softmax')])

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    history = model.fit(train_generator,validation_data=validation_generator,
                        callbacks=[es,mc],
                        epochs=5,verbose=1)
    text = ["Model : {} \n".format(key)]
    for metric in history.history:
      text.append("\t{} : {}\n".format(metric,history.history[metric]))
    results = model.evaluate(test_generator,return_dict=True)
    text.append('\n Test Dataset:\n')
    for metric in results:
      text.append("\t{} : {}\n".format(key,results[metric]))
    file.writelines(text)
    file.close()
    gc.collect()
    del model

train_multiple_models(cnn_models)

In [6]:
if not os.path.exists('/content/gdrive/My Drive/Neeraj_temp'+'/'+'05_11_2020'):
  os.mkdir('/content/gdrive/My Drive/Neeraj_temp'+'/'+'05_11_2020')
path = '/content/gdrive/My Drive/Neeraj_temp'+'/'+'05_11_2020'
key = 'Mobilenet'
clear_session()

base_dataset_path = '/content/content/Modified_Dataset_224_gray' # (20,224,224)
#base_dataset_path = '/content/content/Modified_Dataset' # (20,90,90,3)
train_generator = custom_generator(base_dataset_path+'/'+"Train",16)
validation_generator = custom_generator(base_dataset_path+'/'+'Validation',16)
test_generator = custom_generator(base_dataset_path+'/'+"Test",16)
log_dir = path+'/'+"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=15)
mc = ModelCheckpoint(path+'/'+'best_model_{}.h5'.format(key), monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

file = open(path+'/'+'Report{}.txt'.format(key),'w')
pretrained = cnn_models[key]
model = Sequential([TimeDistributed(pretrained),
                    TimeDistributed(Flatten()),
                    # LSTMS - Recurrent Network Layer
                    #Bidirectional(LSTM(32, return_sequences=True)),
                    #LSTM(32, return_sequences=True),
                    LSTM(32),
                    Dropout(0.5),
                    Flatten(),

                    # # FC layers group
                    Dense(2048, activation='relu'),
                    Dropout(.5),
                    Dense(1024, activation='relu'),
                    Dropout(.5),
                    Dense(20, activation='softmax')])

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
history = model.fit(train_generator,validation_data=validation_generator,
                    callbacks=[es,mc],workers=4,
                    epochs=25,verbose=1)
text = ["Model : {} \n".format(key)]
for metric in history.history:
  text.append("\t{} : {}\n".format(metric,history.history[metric]))
results = model.evaluate(test_generator,return_dict=True)
text.append('\n Test Dataset:\n')
for metric in results:
  text.append("\t{} : {}\n".format(key,results[metric]))
file.writelines(text)
file.close()
gc.collect()
del model


NameError: ignored

In [ ]:
%ten

{'accuracy': [0.07249999791383743],
 'loss': [2.875523090362549],
 'val_accuracy': [0.05249999836087227],
 'val_loss': [2.9540960788726807]}

In [ ]:
!cp -r ./cropped '/content/gdrive/My Drive/new_cropped'

^C


In [12]:
def get_size(start_path = '.', unit='MB'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    if unit == 'MB':
      total_size *= 1e-6
    elif unit == 'GB':
      total_size *= 1e-9
    elif unit == 'TB':
      total_size *= 1e-12

    return total_size

target_dir = "/content/content" #@param {type: 'string'}
output_unit = 'GB' #@param ['MB', 'GB', 'TB']

dir_size = round(get_size(target_dir, output_unit), 2)
print('Target directory:', target_dir)
print('Directory size is', dir_size, output_unit)

Target directory: /content/content
Directory size is 11.53 GB


In [5]:
import time
base_dataset_path = '/content/content/Modified_Dataset_224_gray' # (20,224,224)
#base_dataset_path = '/content/content/Modified_Dataset' # (20,90,90,3)
start = time.time()
train_generator = custom_generator(base_dataset_path+'/'+"Train",32)
print('Constructor time : ',time.time()-start)

start = time.time()
train_generator[0]
print('Batch_time : ',time.time()-start)

Constructor time :  0.03332018852233887
Batch_time :  9.411264419555664
